In [1]:
import numpy as np
import random
from fractions import Fraction
import re
import time

In [2]:
class EnhancedGeneticEquationSolver:
    def __init__(self, pop_size=500, generations=5000, mutation_rate=0.2, elite_size=50,
                 var_range=(-100, 100), tournament_size=5, stagnation_limit=200, 
                 adaptive_mutation=True, fraction_precision=True):
        """
        Enhanced genetic algorithm for solving equation systems
        
        pop_size: Population size (increased)
        generations: Maximum number of generations
        mutation_rate: Base mutation rate
        elite_size: Number of elite solutions to preserve
        var_range: Range of variable values
        tournament_size: Tournament size for selection
        stagnation_limit: Max generations without improvement before restart
        adaptive_mutation: Whether to use adaptive mutation rates
        fraction_precision: Whether to bias toward common fractions
        """
        self.pop_size = pop_size
        self.generations = generations
        self.base_mutation_rate = mutation_rate
        self.current_mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.var_range = var_range
        self.tournament_size = tournament_size
        self.stagnation_limit = stagnation_limit
        self.adaptive_mutation = adaptive_mutation
        self.fraction_precision = fraction_precision
        # Common fractions to bias toward (to improve exact fraction finding)
        self.common_fractions = [
            Fraction(1, i) for i in range(1, 21)  # 1/1, 1/2, 1/3, ... 1/20
        ] + [Fraction(i, j) for i in range(-20, 21) for j in range(1, 21)
             if i != 0 and abs(i) <= abs(j)]  # Common fractions like 1/2, 2/3, 3/4, etc.
        
    def fitness_function(self, chromosome, equations):
        """
        Calculate fitness based on how well the chromosome solves all equations
        
        chromosome: Chromosome (variable values)
        equations: Equation functions
        :return: Fitness score (higher is better)
        """
        total_error = 0
        
        for eq in equations:
            try:
                result = eq(*chromosome)
                total_error += result**2  # Use squared error for better convergence
            except (ZeroDivisionError, ValueError, OverflowError, TypeError):
                # Heavily penalize invalid solutions
                total_error += 1e6
        
        # Avoid division by zero
        if total_error < 1e-15:
            return float('inf')
        
        return 1.0 / total_error
    
    def initialize_population(self, var_count):
        """
        Create diverse initial population with different strategies
        
        var_count: Number of variables
        :return: Initial population
        """
        population = []
        
        # Add solutions with integer values
        for _ in range(self.pop_size // 4):
            chromosome = [random.randint(self.var_range[0], self.var_range[1]) for _ in range(var_count)]
            # Avoid zeros if possible
            chromosome = [1 if gene == 0 else gene for gene in chromosome]
            population.append(chromosome)
            
        # Add solutions with small integer values (higher probability of being correct)
        for _ in range(self.pop_size // 4):
            chromosome = [random.randint(-10, 10) for _ in range(var_count)]
            chromosome = [1 if gene == 0 else gene for gene in chromosome]
            population.append(chromosome)
        
        # Add solutions with common fractions
        for _ in range(self.pop_size // 4):
            chromosome = []
            for _ in range(var_count):
                if random.random() < 0.7:  # 70% chance of common fraction
                    gene = float(random.choice(self.common_fractions))
                else:
                    gene = random.uniform(self.var_range[0], self.var_range[1])
                chromosome.append(gene)
            population.append(chromosome)
            
        # Add solutions with small decimals
        for _ in range(self.pop_size - len(population)):
            chromosome = [random.uniform(-10, 10) for _ in range(var_count)]
            chromosome = [0.1 if abs(gene) < 1e-10 else gene for gene in chromosome]
            population.append(chromosome)
            
        return population
    
    def evaluate_population(self, population, equations):
        """        
        population: Population of chromosomes
        equations: Equation functions
        :return: Ranked population with fitness scores
        """
        fitness_scores = []
        for chromosome in population:
            fitness = self.fitness_function(chromosome, equations)
            fitness_scores.append((chromosome, fitness))
        
        return sorted(fitness_scores, key=lambda x: x[1], reverse=True)
    
    def selection(self, ranked_population):
        """        
        ranked_population: Ranked population
        :return: Selected chromosome
        """
        tournament = random.sample(ranked_population, self.tournament_size)
        # Select the best with high probability, but sometimes take second best
        if random.random() < 0.9:
            return max(tournament, key=lambda x: x[1])[0]
        else:
            # Sort tournament and take second best if available
            sorted_tournament = sorted(tournament, key=lambda x: x[1], reverse=True)
            if len(sorted_tournament) > 1:
                return sorted_tournament[1][0]
            return sorted_tournament[0][0]
    
    def crossover(self, parent1, parent2):
        """        
        parent1: First parent
        parent2: Second parent
        :return: Child chromosome
        """
        # Choose crossover strategy randomly
        strategy = random.choice(['uniform', 'arithmetic', 'single_point'])
        
        if strategy == 'uniform':
            # Uniform crossover (each gene from either parent with 50% chance)
            child = [p1 if random.random() < 0.5 else p2 for p1, p2 in zip(parent1, parent2)]
            
        elif strategy == 'arithmetic':
            # Arithmetic crossover (weighted average of parents)
            alpha = random.random()  # Random weight
            child = [alpha * p1 + (1 - alpha) * p2 for p1, p2 in zip(parent1, parent2)]
            
        else:  # single_point
            point = random.randint(1, len(parent1) - 1)
            child = parent1[:point] + parent2[point:]
            
        return child
    
    def mutation(self, chromosome, generation, max_generations):
        """        
        chromosome: Chromosome to mutate
        generation: Current generation number
        max_generations: Maximum generations
        :return: Mutated chromosome
        """
        mutated = chromosome.copy()
        
        # Adjust mutation rate if adaptive mutation is enabled
        if self.adaptive_mutation:
            # Start with higher mutation rate, then decrease
            progress = min(1.0, generation / (max_generations * 0.7))
            self.current_mutation_rate = self.base_mutation_rate * (1.0 - 0.6 * progress)
            
            # Increase if population seems stuck (handled in solve method)
        
        for i in range(len(mutated)):
            # Each gene mutates with current rate
            if random.random() < self.current_mutation_rate:
                strategy = random.choices(
                    ['random', 'small_change', 'fraction', 'zero', 'sign_flip'],
                    weights=[0.3, 0.4, 0.2, 0.05, 0.05],
                    k=1
                )[0]
                
                if strategy == 'random':
                    # Completely new random value
                    mutated[i] = random.uniform(self.var_range[0], self.var_range[1])
                    
                elif strategy == 'small_change':
                    # Small adjustment to current value
                    magnitude = abs(mutated[i]) * 0.1 if mutated[i] != 0 else 0.1
                    mutated[i] += random.uniform(-magnitude, magnitude)
                    
                elif strategy == 'fraction' and self.fraction_precision:
                    # Set to a common fraction
                    mutated[i] = float(random.choice(self.common_fractions))
                    
                elif strategy == 'zero':
                    # Set close to zero (not exactly to avoid division errors)
                    mutated[i] = random.choice([0.1, -0.1])
                    
                elif strategy == 'sign_flip':
                    # Flip sign
                    mutated[i] = -mutated[i]
        
        return mutated
    
    def next_generation(self, current_ranked_population, generation, max_generations):
        """        
        current_ranked_population: Current ranked population
        generation: Current generation number
        max_generations: Maximum generations
        :return: New population
        """
        # Get elite solutions
        elites = [x[0] for x in current_ranked_population[:self.elite_size]]
        
        # Create children using selection, crossover and mutation
        children = []
        while len(children) < self.pop_size - len(elites):
            # Select parents
            parent1 = self.selection(current_ranked_population)
            parent2 = self.selection(current_ranked_population)
            
            # Create child via crossover
            child = self.crossover(parent1, parent2)
            
            # Mutate child
            child = self.mutation(child, generation, max_generations)
            
            children.append(child)
        
        # New population with elites and children
        new_population = elites + children
        return new_population
    
    def restart_population(self, best_solution, var_count):
        """        
        best_solution: Best solution found so far
        var_count: Number of variables
        :return: New diverse population
        """
        # Initialize new population
        new_population = self.initialize_population(var_count)
        
        # Replace some individuals with variations of best solution
        num_seeds = min(self.pop_size // 10, 50)  # 10% or up to 50 individuals
        
        for i in range(num_seeds):
            # Create variation of best solution
            variation = best_solution.copy()
            
            # Apply small mutations to create diversity
            for j in range(var_count):
                if random.random() < 0.3:  # 30% chance to mutate each gene
                    magnitude = abs(variation[j]) * 0.1 if variation[j] != 0 else 0.1
                    variation[j] += random.uniform(-magnitude, magnitude)
            
            # Replace random individual in population
            new_population[random.randint(0, self.pop_size - 1)] = variation
        
        return new_population
    
    def approximate_fractions(self, solution, max_denominator=100):
        """        
        solution: Solution as floats
        max_denominator: Maximum denominator
        :return: Solution as fractions
        """
        result = []
        for value in solution:
            # Try to find closest common fraction first
            if self.fraction_precision:
                best_fraction = None
                min_diff = float('inf')
                
                for frac in self.common_fractions:
                    diff = abs(float(frac) - value)
                    if diff < min_diff and diff < 1e-2:  # Within reasonable tolerance
                        min_diff = diff
                        best_fraction = frac
                
                if best_fraction is not None:
                    result.append(best_fraction)
                    continue
            
            # Fall back to standard fraction approximation
            fraction = Fraction(value).limit_denominator(max_denominator)
            result.append(fraction)
        
        return result
    
    def solve(self, equations, var_count):
        """        
        equations: Equation functions
        var_count: Number of variables
        :return: Best solution as fractions
        """
        start_time = time.time()
        
        # Initialize
        population = self.initialize_population(var_count)
        best_solution = None
        best_fitness = -float('inf')
        generations_without_improvement = 0
        restart_count = 0
        
        for generation in range(self.generations):
            # Evaluate population
            ranked_population = self.evaluate_population(population, equations)
            
            # Check best solution
            current_best = ranked_population[0]
            current_fitness = current_best[1]
            
            if current_fitness > best_fitness:
                best_solution = current_best[0]
                best_fitness = current_fitness
                generations_without_improvement = 0
                
                # If exact solution found, break early
                if best_fitness > 1e10:
                    print(f"Exact solution found in generation {generation}.")
                    break
            else:
                generations_without_improvement += 1
            
            # Report progress
            if generation % 100 == 0 or generation == self.generations - 1:
                error = 1.0 / best_fitness if best_fitness != float('inf') else 0
                elapsed_time = time.time() - start_time
                print(f"Generation {generation}: Best error = {error:.10f}, Time: {elapsed_time:.2f}s")
            
            # Check if population is stuck
            if generations_without_improvement >= self.stagnation_limit:
                if restart_count < 5:  # Limit number of restarts
                    # Restart population but keep best solution
                    print(f"Population stuck for {generations_without_improvement} generations. Restarting...")
                    population = self.restart_population(best_solution, var_count)
                    generations_without_improvement = 0
                    restart_count += 1
                    
                    # Increase mutation rate temporarily
                    if self.adaptive_mutation:
                        self.current_mutation_rate = min(0.5, self.base_mutation_rate * 2)
                else:
                    # If we've restarted too many times without success, try to refine best solution
                    print("Multiple restarts without significant improvement. Focusing on refinement...")
                    # Focus on small changes to best solution
                    population = []
                    for _ in range(self.pop_size):
                        variation = best_solution.copy()
                        for i in range(len(variation)):
                            # Small random adjustments
                            if random.random() < 0.3:
                                magnitude = abs(variation[i]) * 0.01 if variation[i] != 0 else 0.01
                                variation[i] += random.uniform(-magnitude, magnitude)
                        population.append(variation)
                    generations_without_improvement = 0
            
            # Create next generation
            population = self.next_generation(ranked_population, generation, self.generations)
        
        # Convert solution to fractions
        solution_fractions = self.approximate_fractions(best_solution)
        
        error = 1.0 / best_fitness if best_fitness != float('inf') else 0
        elapsed_time = time.time() - start_time
        print(f"Best solution: {solution_fractions}")
        print(f"Final error: {error:.10f}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        
        return solution_fractions

In [3]:
def parse_advanced_equation(equation_str, var_names):
    """
    Convert an equation string to a function
    
    :param equation_str: Equation string
    :param var_names: Variable names
    :return: Function that returns the error
    """
    # Split left and right sides
    sides = equation_str.replace(" ", "").split("=")
    left_side = sides[0]
    right_side = sides[1]
    
    # Create error function
    def equation_error(*vars):
        # Create variable dictionary
        var_dict = {name: value for name, value in zip(var_names, vars)}
        
        try:
            # Calculate both sides of the equation
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            # Return absolute error
            return abs(left_val - right_val)
        except (ZeroDivisionError, ValueError, OverflowError):
            # Return large error
            return 1e6
    
    return equation_error


def parse_equations_system(equations_str, var_names):
    """
    Convert equation strings to functions
    
    :param equations_str: List of equation strings
    :param var_names: Variable names
    :return: List of error functions
    """
    equation_functions = []
    for eq_str in equations_str:
        equation_functions.append(parse_advanced_equation(eq_str, var_names))
    return equation_functions


def print_fraction(frac):
    """Display fraction in readable format"""
    if frac.denominator == 1:
        return str(frac.numerator)
    else:
        return f"{frac.numerator}/{frac.denominator}"


def solve_equations(equations, var_names):
    """Solve equation system"""
    # Convert equations to functions
    equations_funcs = parse_equations_system(equations, var_names)
    
    solver = EnhancedGeneticEquationSolver(
        pop_size=500,             
        generations=1000,         
        mutation_rate=0.2,      
        elite_size=50,          
        var_range=(-100, 100),    
        tournament_size=5,      
        stagnation_limit=200,     
        adaptive_mutation=True,   
        fraction_precision=True   
    )
    
    solution = solver.solve(equations_funcs, len(var_names))
    
    print("\nFinal solution:")
    for var, val in zip(var_names, solution):
        print(f"{var} = {print_fraction(val)}")
    
    print("\nVerifying solution in equations:")
    for i, eq_str in enumerate(equations):
        try:
            # Create variable dictionary
            var_dict = {name: float(value) for name, value in zip(var_names, solution)}
            
            # Calculate both sides
            sides = eq_str.replace(" ", "").split("=")
            left_side = sides[0]
            right_side = sides[1]
            
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            print(f"Equation {i+1}: {left_val} ≈ {right_val}, error: {abs(left_val - right_val)}")
        except Exception as e:
            print(f"Equation {i+1}: Error in verification - {str(e)}")
    
    return solution


def test_known_solution(equations, var_names, known_solution):
    """Test the known solution in the equations"""
    print("\n=== Validating Known Solution ===")
    
    # Display known solution
    print("Known solution:")
    for var, val in zip(var_names, known_solution):
        print(f"{var} = {print_fraction(val)}")
    
    # Verify in equations
    print("\nVerifying in equations:")
    for i, eq_str in enumerate(equations):
        try:
            # Create variable dictionary
            var_dict = {name: float(value) for name, value in zip(var_names, known_solution)}
            
            # Calculate both sides
            sides = eq_str.replace(" ", "").split("=")
            left_side = sides[0]
            right_side = sides[1]
            
            left_val = eval(left_side, {"__builtins__": {}}, var_dict)
            right_val = eval(right_side, {"__builtins__": {}}, var_dict)
            
            print(f"Equation {i+1}: {left_val} ≈ {right_val}, error: {abs(left_val - right_val)}")
        except Exception as e:
            print(f"Equation {i+1}: Error in verification - {str(e)}")


def solve_2var_system():
    """Solve the 2-variable equation system from the image"""
    print("\n=== Solving 2-variable equation system ===")
    
    equations = [
        "x + 2*y = 4",
        "4*y + 4*x= 12",
    ]
    
    var_names = ['x', 'y']
    
    # Known solution from image
    known_solution = [
        Fraction(2, 1),
        Fraction(1, 1)
    ]
    
    # Verify known solution
    test_known_solution(equations, var_names, known_solution)
    
    # Solve with enhanced genetic algorithm
    solve_equations(equations, var_names)

def solve_3var_system():
    """Solve the 3-variable equation system from the image"""
    print("\n=== Solving 3-variable equation system ===")
    
    equations = [
        "6*x - 2*y + 8*z = 20",
        "y + 8*x * z = -1",
        "2*z * (6/x) + (3/2)*y = 6"
    ]
    
    var_names = ['x', 'y', 'z']
    
    # Known solution from image
    known_solution = [
        Fraction(2, 3),   # x = 2/3
        Fraction(-5, 1),  # y = -5
        Fraction(3, 4)    # z = 3/4
    ]
    
    # Verify known solution
    test_known_solution(equations, var_names, known_solution)
    
    # Solve with enhanced genetic algorithm
    solve_equations(equations, var_names)

def solve_4var_system():
    """Solve the 4-variable system"""
    print("\n=== Solving 4-variable equation system ===")
    
    equations = [
        "(1/15)*x - 2*y - 15*z - (4/5)*t = 3",
        "-(5/2)*x - (9/4)*y + 12*z - t = 17",
        "-13*x + (3/10)*y - 6*z - (2/5)*t = 17",
        "(1/2)*x + 2*y + (7/4)*z + (4/3)*t = -9"
    ]
    
    var_names = ['x', 'y', 'z', 't']
    
    # Known solution from image
    known_solution = [
        Fraction(-3, 2),  # x = -3/2
        Fraction(-7, 2),  # y = -7/2
        Fraction(1, 3),   # z = 1/3
        Fraction(-11, 8)  # t = -11/8
    ]
    
    test_known_solution(equations, var_names, known_solution)
    
    solve_equations(equations, var_names)


solve_2var_system()
solve_3var_system()
solve_4var_system()


=== Solving 2-variable equation system ===

=== Validating Known Solution ===
Known solution:
x = 2
y = 1

Verifying in equations:
Equation 1: 4.0 ≈ 4, error: 0.0
Equation 2: 12.0 ≈ 12, error: 0.0
Generation 0: Best error = 4.0000000000, Time: 0.03s
Exact solution found in generation 22.
Best solution: [Fraction(2, 1), Fraction(1, 1)]
Final error: 0.0000000001
Total time: 0.83 seconds

Final solution:
x = 2
y = 1

Verifying solution in equations:
Equation 1: 4.0 ≈ 4, error: 0.0
Equation 2: 12.0 ≈ 12, error: 0.0

=== Solving 3-variable equation system ===

=== Validating Known Solution ===
Known solution:
x = 2/3
y = -5
z = 3/4

Verifying in equations:
Equation 1: 20.0 ≈ 20, error: 0.0
Equation 2: -1.0 ≈ -1, error: 0.0
Equation 3: 6.0 ≈ 6, error: 0.0
Generation 0: Best error = 126.0342298175, Time: 0.04s
Exact solution found in generation 35.
Best solution: [Fraction(2, 3), Fraction(-5, 1), Fraction(3, 4)]
Final error: 0.0000000001
Total time: 1.79 seconds

Final solution:
x = 2/3
y = 